## Задание 1

In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
warnings.simplefilter("ignore")
%matplotlib inline

In [2]:
data = pd.read_csv(
    "data.csv"
)
data.head(n=2)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
print(len(data))

100000


In [3]:
data.isnull().sum().sum()
#data.head(n=2)

15550955

In [6]:
df = data.dropna(axis=1)

In [7]:
df.shape

(100000, 75)

In [8]:
## Задание 2

In [9]:
df = data[['TransactionAmt','isFraud', 'card1', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1']]
df

,TransactionAmt,isFraud,card1,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1
0,68.500,0,13926,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0
1,29.000,0,2755,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,59.000,0,4663,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
3,50.000,0,18132,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0
4,50.000,0,4497,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,55.385,0,2256,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
99996,117.000,0,2518,2.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,14.0
99997,50.000,0,2748,1.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,25.0
99998,100.000,0,16075,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0


In [10]:
x_train, x_test = train_test_split(
    df.drop(["isFraud"], axis=1), train_size=0.7, shuffle=True, random_state=1,
)
y_train, y_test = train_test_split(
    df["isFraud"], train_size=0.7, shuffle=True, random_state=1,
)
print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_test.shape = {} rows, {} cols".format(*x_test.shape))

x_train.shape = 70000 rows, 17 cols
x_test.shape = 30000 rows, 17 cols


In [11]:
pipeline = Pipeline(
    steps=[
        ("scaling", StandardScaler()),
        ("model", LogisticRegression(random_state=27))
    ]
)

In [13]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('scaling', StandardScaler()),
                ('model', LogisticRegression(random_state=27))])

In [14]:
train_score = roc_auc_score(y_train, pipeline.predict_proba(x_train)[:, 1])
test_score = roc_auc_score(y_test, pipeline.predict_proba(x_test)[:, 1])

print(f"Train-score: {round(train_score, 3)}, Test-score: {round(test_score, 3)}")

Train-score: 0.71, Test-score: 0.709


In [15]:
#Задание 3

x_train, x_valid = train_test_split(
    df.drop(["isFraud"], axis=1), train_size=0.5, shuffle=True, random_state=1,
)
y_train, y_valid = train_test_split(
    df["isFraud"], train_size=0.5, shuffle=True, random_state=1,
)

x_valid, x_test = train_test_split(
    x_valid, train_size=0.5, shuffle=True, random_state=1
)
y_valid, y_test = train_test_split(
    y_valid, train_size=0.5, shuffle=True, random_state=1
)

print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))
print("x_test.shape = {} rows, {} cols".format(*x_test.shape))

x_train.shape = 50000 rows, 17 cols
x_valid.shape = 25000 rows, 17 cols
x_test.shape = 25000 rows, 17 cols


In [16]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('scaling', StandardScaler()),
                ('model', LogisticRegression(random_state=27))])

In [21]:
train_score = roc_auc_score(y_train, pipeline.predict_proba(x_train)[:, 1])
valid_score = roc_auc_score(y_valid, pipeline.predict_proba(x_valid)[:, 1])
test_score = roc_auc_score(y_test, pipeline.predict_proba(x_test)[:, 1])

print(f"Train-score: {round(train_score, 4)}, Valid-score: {round(valid_score, 4)}, Test-score: {round(test_score, 4)}")

Train-score: 0.7073, Valid-score: 0.7064, Test-score: 0.7181


In [22]:
test_score - valid_score

0.011685838266002557

In [23]:
#Задание 4/5

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=27)

# проводим кросс-валидацию
cv = cross_val_score(
    estimator=pipeline,
    X=df.drop(["isFraud"], axis=1),
    y=df["isFraud"],
    scoring="roc_auc",
    cv=kfold
)

# Считаем среднее значение метрики на каждом фолде и выводим среднее значение
print(f"CV-results: {round(np.mean(cv), 4)} +/- {round(np.std(cv), 3)}")

CV-results: 0.6901 +/- 0.012


Мы получили более устойчивые результаты, которые позволят более честно оценить качество модели и использовать текцщее решенеи, как отправную точку для дальнейшего улучшения решения. Проблема нашего подхода заключается в том, что мы используем не все данные для обучения и для тестирования модели. Возможно, что разбиение, которое мы провели является слишком специфичным и не позволяет оценить качество модели объективно. 